### Check all seasons:

In [17]:
import pandas as pd

# --- Load files ---
sf_df = pd.read_excel("New Assets Report - Gil-2025-08-19-02-20-51.xlsx")
robo_df = pd.read_excel("Season_Tickets_modified.xlsx")

In [18]:
# --- Sums instead of counts ---
sf_sums = (
    sf_df.groupby("Asset Name")
         .agg(
             SF_Count=("Asset Name", "size"),
             SF_Price_Sum=("Price", "sum"),
             SF_Prices=("Price", lambda s: sorted(pd.Series(s).dropna().unique()))
         )
         .reset_index()
)

robo_sums = (
    robo_df.groupby("Asset Name")
           .agg(
               Robo_Count=("Asset Name", "size"),
               Robo_Price_Sum=("Price", "sum"),
               Robo_Prices=("Price", lambda s: sorted(pd.Series(s).dropna().unique()))
           )
           .reset_index()
)

# --- Merge ---
compare_sums = (
    robo_sums
    .merge(sf_sums, on="Asset Name", how="outer")
    .fillna({"Robo_Count": 0, "SF_Count": 0, "Robo_Price_Sum": 0, "SF_Price_Sum": 0})
)

# Cast numeric columns
for col in ["Robo_Count", "SF_Count"]:
    compare_sums[col] = compare_sums[col].astype(int)

for col in ["Robo_Price_Sum", "SF_Price_Sum"]:
    compare_sums[col] = compare_sums[col].astype(float)

# Ensure lists are lists (not NaN)
for col in ["Robo_Prices", "SF_Prices"]:
    compare_sums[col] = compare_sums[col].apply(lambda v: v if isinstance(v, list) else [])

compare_sums['Robo_Price_Sum'] = compare_sums['Robo_Price_Sum'].astype(int)
compare_sums['SF_Price_Sum'] = compare_sums['SF_Price_Sum'].astype(int)

# --- Price match check (unique price sets) ---
compare_sums["Price_Match"] = compare_sums.apply(
    lambda r: set(r["Robo_Prices"]) == set(r["SF_Prices"]), axis=1
)

# --- Final order ---
compare_sums = compare_sums[[
    "Asset Name",
    "Robo_Count", "SF_Count",
    "Robo_Price_Sum", "SF_Price_Sum",
    "Price_Match"
]]

compare_sums


,Asset Name,Robo_Count,SF_Count,Robo_Price_Sum,SF_Price_Sum,Price_Match
0,מנוי לעונת 2014-2015,3922,3922,4171177,4171177,True
1,מנוי לעונת 2015-2016,5793,5793,7648027,7648027,True
2,מנוי לעונת 2016-2017,6534,6534,10062290,10062290,True
3,מנוי לעונת 2017-2018,6944,6944,11427902,11427902,True
4,מנוי לעונת 2018-2019,5792,5792,9370745,9370745,True
5,מנוי לעונת 2019-2020,5213,5213,9011789,9011789,True
6,מנוי לעונת 2021-2022,2809,2809,4708304,4708304,True
7,מנוי לעונת 2022-2023,3535,3535,5378378,5378378,False
8,מנוי לעונת 2023-2024,4256,3396,7046595,4255450,False
9,מנוי לעונת 2024/2025,0,13,0,16500,False


In [19]:
4256-3396

860